In [1]:
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from scipy.signal import get_window
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time


In [2]:
n_fft = 2048
hop_length= int(n_fft/4)
window ='hann'
Begin_bin= 0
frequency_bins = 112
# window = get_window('flattop', n_fft)

## Check 

In [3]:
# file_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/train/Queen_train/queen_8.wav'
# y, sr = librosa.load(file_path)
# stft_result = librosa.stft(y, n_fft=n_fft, hop_length=512, window=window)
# magnitude = np.abs(stft_result)
# magnitude_db = librosa.amplitude_to_db(magnitude)
# df = pd.DataFrame(magnitude_db[:40, :]) 

# plt.figure(figsize=(10,4))
# librosa.display.specshow(magnitude_db,sr=sr,x_axis='time',y_axis= 'hz')
# plt.colorbar(format='%2.0f dB')
# plt.title('STFT Magnitude')
# plt.show()

In [4]:
# file_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/train/Queen_train/queen_0.wav'
# y, sr = librosa.load(file_path)
# stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
# magnitude = np.abs(stft_result)
# magnitude_db = librosa.amplitude_to_db(magnitude)

# frequencies = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
# df = pd.DataFrame(magnitude_db[:frequency_bins, :])

# df.insert(0, 'Hz', frequencies[:frequency_bins])
# df


In [5]:

# df_trans = df.mean(axis=1).to_frame().T
# df_trans

In [6]:

# df_trans = df.std(axis=1).to_frame().T
# df_trans

##  Queen Train

In [7]:
folder_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/train/Queen_train/'

wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

df_merge_train_queen = pd.DataFrame()
df_merge_train_queen_mean= pd.DataFrame()
df_merge_train_queen_std= pd.DataFrame()

for wav_file in wav_files:
    file_path = os.path.join(folder_path, wav_file)
    
    y, sr = librosa.load(file_path)
    stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    magnitude = np.abs(stft_result)
    magnitude_db = librosa.amplitude_to_db(magnitude)

    df = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df2 = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df_trans = df.std(axis=1).to_frame().T
    df_trans2 = df2.mean(axis=1).to_frame().T
    
    df_merge_train_queen_mean = pd.concat([df_merge_train_queen_mean,df_trans2], axis=0, ignore_index=True)
    df_merge_train_queen_std = pd.concat([df_merge_train_queen_std,df_trans], axis=0, ignore_index=True)
    df_merge_train_queen = pd.concat([df_merge_train_queen_mean, df_merge_train_queen_std], axis=1, ignore_index=True)

df_merge_train_queen['Label'] = 1
df_merge_train_queen


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-22.808546,-12.654547,-7.936298,-4.086808,-7.046385,-7.348252,-8.480138,-12.318094,-10.287766,-11.028598,...,7.051106,5.280563,5.997036,5.997731,5.639641,5.453162,4.922418,6.231715,7.037271,1
1,-26.806057,-18.806309,-14.124155,-14.529056,-11.003247,-7.421661,-8.109457,-14.549718,-14.467135,-12.110363,...,7.352593,6.003704,6.612796,6.436133,6.738337,5.679572,6.822301,7.162236,7.738001,1
2,-26.268703,-11.388426,-8.909139,-0.057067,3.245238,-2.529802,-8.362550,-15.317734,-16.868231,-15.764844,...,5.737398,6.732986,6.590871,5.680191,7.290667,4.596172,5.662388,5.284963,5.673373,1
3,-22.193598,-13.884340,-12.828590,-13.436923,-13.248388,-11.868794,-13.194749,-16.601700,-16.690025,-12.916192,...,7.727414,8.085557,6.639987,7.349772,6.911750,8.228676,6.640068,7.316347,8.060115,1
4,-27.683928,-20.607801,-15.193899,-6.478353,-8.962743,-10.280721,-9.714802,-13.906053,-14.584490,-11.986362,...,5.749352,5.628262,4.792911,5.209894,5.063713,6.053167,4.986611,5.490445,6.404570,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,-12.635983,-1.609449,3.597828,6.292580,10.351928,8.823968,6.529092,5.708021,7.089710,7.292682,...,5.969552,5.821225,5.778186,6.765182,6.324294,5.525216,5.954312,5.214452,5.320183,1
6996,-11.699414,-2.354914,1.468952,6.540047,8.581634,9.167048,6.366939,3.582814,4.534622,8.769086,...,6.206879,5.984836,6.055716,5.414366,6.052112,5.552516,4.612333,5.352575,6.268008,1
6997,-27.383867,-17.003410,-12.413963,-10.684532,-5.866035,-7.248629,-10.265894,-9.604950,-10.877103,-11.243253,...,5.409110,4.637899,5.441229,6.587791,5.391552,5.220777,5.606618,6.530772,6.614305,1
6998,-13.090653,-2.738225,0.515090,3.844620,7.900819,7.528256,3.956966,3.820188,2.736565,3.296423,...,5.056871,6.031544,5.531102,5.423471,6.440923,6.171616,5.798957,5.712042,6.266681,1


##  Not Queen Train

In [8]:
folder_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/train/NonQueen_train/'

wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

df_merge_train_not_queen = pd.DataFrame()
df_merge_train_not_queen_mean= pd.DataFrame()
df_merge_train_not_queen_std= pd.DataFrame()

for wav_file in wav_files:
    file_path = os.path.join(folder_path, wav_file)
    
    y, sr = librosa.load(file_path)
    stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    magnitude = np.abs(stft_result)
    magnitude_db = librosa.amplitude_to_db(magnitude)

    df = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df2 = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df_trans = df.std(axis=1).to_frame().T
    df_trans2 = df2.mean(axis=1).to_frame().T
    
    df_merge_train_not_queen_mean = pd.concat([df_merge_train_not_queen_mean,df_trans2], axis=0, ignore_index=True)
    df_merge_train_not_queen_std = pd.concat([df_merge_train_not_queen_std,df_trans], axis=0, ignore_index=True)
    df_merge_train_not_queen = pd.concat([df_merge_train_not_queen_mean, df_merge_train_not_queen_std], axis=1, ignore_index=True)

df_merge_train_not_queen['Label'] = 0
df_merge_train_not_queen


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-6.079378,1.977884,6.964907,9.078245,9.976440,12.005538,13.232248,13.379407,13.229007,15.423858,...,6.012274,5.277337,5.016967,5.930026,5.219456,4.540161,4.295200,4.843753,4.705482,0
1,-15.088403,-3.049372,1.622162,2.414374,3.679789,3.685953,4.174804,2.562248,0.817477,2.639108,...,6.045747,5.651623,5.709354,5.965483,5.264354,5.096726,5.989956,5.222790,5.440136,0
2,-30.904833,-22.036747,-16.578165,-10.816679,-6.596766,-7.481753,-13.040244,-14.606665,-15.350325,-13.523809,...,6.666417,8.166783,4.206422,5.975694,5.851607,5.930387,6.016934,5.834573,5.713196,0
3,-28.638243,-19.916800,-16.756697,-8.917939,-7.320681,-12.080371,-13.475293,-8.626266,-10.599179,-9.780050,...,6.160369,6.264543,6.087537,6.555973,6.049340,5.707383,5.313642,5.716227,6.242299,0
4,-29.919601,-19.438465,-14.251174,-8.988425,-6.459613,-12.415199,-14.410398,-12.874078,-13.802051,-14.132752,...,4.711007,5.563325,7.208050,6.218544,5.893248,5.573750,5.662785,5.432616,6.486081,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,-27.066959,-15.808598,-12.016234,-8.840290,-5.700643,-6.064118,-7.791044,-12.309853,-11.117221,-8.813871,...,5.235035,5.938197,5.658187,5.643641,5.664616,4.293982,5.344193,6.508696,7.312350,0
6996,-28.591917,-16.455128,-10.218955,-7.609904,-2.227705,-3.224565,-9.300794,-9.731773,-9.796335,-8.196024,...,5.934849,5.620207,6.016663,6.254725,5.827891,7.081559,6.435751,5.140052,6.828896,0
6997,-29.492817,-19.733240,-14.214334,-9.887540,-7.774919,-8.535912,-11.585422,-10.739936,-9.519459,-7.752552,...,6.910249,5.747013,5.114785,5.561242,5.631642,7.342589,6.485736,5.823280,6.469854,0
6998,-29.936190,-19.070801,-12.754254,-8.965271,-5.139182,-5.019722,-7.255493,-10.159325,-9.912843,-9.278399,...,5.590568,4.949354,5.770982,5.942149,5.606650,5.005726,5.375594,5.737081,5.174795,0


## Queen Val

In [9]:
folder_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/val/Queen/'

wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

df_merge_val_queen = pd.DataFrame()
df_merge_val_queen_mean= pd.DataFrame()
df_merge_val_queen_std= pd.DataFrame()

for wav_file in wav_files:
    file_path = os.path.join(folder_path, wav_file)
    
    y, sr = librosa.load(file_path)
    stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    magnitude = np.abs(stft_result)
    magnitude_db = librosa.amplitude_to_db(magnitude)

    df = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df2 = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df_trans = df.std(axis=1).to_frame().T
    df_trans2 = df2.mean(axis=1).to_frame().T
    
    df_merge_val_queen_mean = pd.concat([df_merge_val_queen_mean,df_trans2], axis=0, ignore_index=True)
    df_merge_val_queen_std = pd.concat([df_merge_val_queen_std,df_trans], axis=0, ignore_index=True)
    df_merge_val_queen = pd.concat([df_merge_val_queen_mean, df_merge_val_queen_std], axis=1, ignore_index=True)

df_merge_val_queen['Label'] = 1
df_merge_val_queen


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-24.662216,-11.791602,-8.518652,0.894805,4.194261,-3.461858,-10.224401,-13.580605,-17.480345,-15.803328,...,6.924873,6.066279,6.026247,6.190271,6.383075,6.405430,4.935861,7.110244,6.242450,1
1,-26.481541,-17.835039,-13.931384,-12.487250,-15.086113,-14.802674,-15.287166,-15.941193,-14.273537,-10.783414,...,5.790999,5.306063,5.855551,6.200583,6.789444,5.956689,5.292241,6.990422,6.687690,1
2,-28.309921,-18.942816,-12.783260,-3.639091,-5.224491,-4.387319,-1.905211,-9.559539,-15.701831,-5.144418,...,6.505320,5.287730,5.751014,5.823422,5.386421,7.411164,6.291250,4.334960,5.293761,1
3,-25.844639,-18.169298,-14.465200,-9.297646,-9.891333,-5.428600,-1.614984,-9.845560,-16.973204,-8.544937,...,5.157794,6.265764,6.146910,5.656040,5.699413,4.515406,5.723453,5.801444,5.028525,1
4,-29.265167,-19.861982,-14.756304,-12.678593,-11.434124,-10.954335,-14.636957,-12.681704,-8.180491,-4.277531,...,6.428152,5.825292,4.525342,6.833611,5.166483,4.882649,5.370390,5.493199,5.363361,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-11.785171,-0.840186,3.396271,7.243251,11.359898,9.625237,6.943479,6.700897,7.626185,6.481133,...,6.335470,6.050679,5.348109,5.277513,5.013733,5.559213,5.803758,4.814065,5.076519,1
996,-9.366529,0.007134,3.638096,6.306519,8.541777,9.561748,6.154743,5.434706,6.801382,5.917808,...,5.266330,5.325099,4.681406,5.916436,5.304453,4.675086,5.341230,5.429980,5.433732,1
997,-17.610531,-5.182034,-1.103572,4.965742,7.540049,6.400363,4.770808,3.754920,4.662338,5.620414,...,5.748651,6.815560,6.520902,5.977524,6.616235,6.689884,7.015735,5.916312,5.883690,1
998,-26.769537,-14.901019,-11.146732,-8.251113,-5.084808,-3.518081,-7.747316,-10.937958,-12.388127,-9.767564,...,5.927715,5.568169,6.078537,7.416717,6.780629,5.852178,5.647035,5.938136,6.051082,1


##  Not Queen Val


In [10]:
folder_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/val/NonQueen/'

wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

df_merge_val_not_queen = pd.DataFrame()
df_merge_val_not_queen_mean= pd.DataFrame()
df_merge_val_not_queen_std= pd.DataFrame()

for wav_file in wav_files:
    file_path = os.path.join(folder_path, wav_file)
    
    y, sr = librosa.load(file_path)
    stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    magnitude = np.abs(stft_result)
    magnitude_db = librosa.amplitude_to_db(magnitude)

    df = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df2 = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df_trans = df.std(axis=1).to_frame().T
    df_trans2 = df2.mean(axis=1).to_frame().T
    
    df_merge_val_not_queen_mean = pd.concat([df_merge_val_not_queen_mean,df_trans2], axis=0, ignore_index=True)
    df_merge_val_not_queen_std = pd.concat([df_merge_val_not_queen_std,df_trans], axis=0, ignore_index=True)
    df_merge_val_not_queen = pd.concat([df_merge_val_not_queen_mean, df_merge_val_not_queen_std], axis=1, ignore_index=True)


df_merge_val_not_queen['Label'] = 0
df_merge_val_not_queen


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-29.711826,-19.283705,-14.775483,-9.337584,-5.633050,-6.772969,-11.761897,-5.368766,3.210441,0.571505,...,6.578604,6.400720,4.829994,4.395748,4.680747,4.900182,5.371431,3.974248,5.426112,0
1,-30.138588,-19.065464,-14.195608,-11.641183,-12.388525,-13.541247,-15.077920,-14.701552,-12.556610,-14.506434,...,6.069660,5.545527,6.981306,5.166717,5.349965,5.849457,5.339776,5.309628,6.179549,0
2,-31.907024,-23.667854,-18.760578,-14.379205,-12.061988,-14.031529,-16.394896,-14.074139,-10.895491,-6.970160,...,5.382723,4.725596,6.561179,6.337353,4.933115,6.635768,5.231870,4.981818,7.441361,0
3,-26.076738,-18.443830,-15.081300,-11.494707,-8.691649,-8.843844,-15.567657,-13.513943,-10.828974,-12.180229,...,5.886572,4.888126,5.618680,6.465976,5.707365,6.428252,4.838579,4.998237,5.241764,0
4,-26.123426,-17.974251,-14.438658,-13.908774,-13.009284,-11.212842,-13.243500,-16.037043,-19.322559,-17.807142,...,5.048376,6.052081,5.281772,5.208494,5.864797,6.625885,5.632199,5.768892,5.919707,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-24.588856,-12.771607,-10.666306,-7.621988,-2.256988,-5.987503,-10.073015,-13.291087,-10.405654,-6.980733,...,4.478648,5.816936,6.110064,5.658631,6.104353,4.769095,5.808397,6.438618,6.694184,0
996,-25.081734,-14.383967,-11.226851,-8.840997,-7.326797,-8.996678,-9.949492,-9.568517,-10.920794,-9.654237,...,5.043159,5.679020,6.324154,5.492128,5.307256,7.155714,6.057665,8.232124,7.800105,0
997,-27.742722,-15.647682,-12.540700,-9.689403,-8.976809,-9.494387,-10.046231,-11.736938,-13.814394,-10.621295,...,6.271636,5.147232,5.372209,4.941226,4.829800,6.009285,5.022900,5.423048,5.661313,0
998,-25.723001,-14.788491,-10.606321,-8.970684,-5.028297,-5.031048,-8.571443,-11.348908,-11.920524,-9.902374,...,5.993001,4.964811,5.397522,6.747658,5.304681,5.070083,5.026395,5.933467,5.963438,0


##  Queen Test

In [11]:
folder_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/test/Queen/'

wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

df_merge_test_queen = pd.DataFrame()
df_merge_test_queen_mean= pd.DataFrame()
df_merge_test_queen_std= pd.DataFrame()

for wav_file in wav_files:
    file_path = os.path.join(folder_path, wav_file)
    
    y, sr = librosa.load(file_path)
    stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    magnitude = np.abs(stft_result)
    magnitude_db = librosa.amplitude_to_db(magnitude)
    
    df = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df2 = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df_trans = df.std(axis=1).to_frame().T
    df_trans2 = df2.mean(axis=1).to_frame().T
    
    df_merge_test_queen_mean = pd.concat([df_merge_test_queen_mean,df_trans2], axis=0, ignore_index=True)
    df_merge_test_queen_std = pd.concat([df_merge_test_queen_std,df_trans], axis=0, ignore_index=True)
    df_merge_test_queen = pd.concat([df_merge_test_queen_mean, df_merge_test_queen_std], axis=1, ignore_index=True)


df_merge_test_queen['Label'] = 1
df_merge_test_queen


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-29.730820,-19.974104,-15.437284,-13.384739,-12.581177,-12.102852,-9.168703,-10.035564,-14.514035,-12.780434,...,5.745146,5.844039,6.168098,3.945764,5.897210,6.911964,7.077823,5.684820,5.587729,1
1,-28.139399,-17.809059,-15.051513,-13.590136,-9.776896,-10.534388,-14.288779,-17.136816,-16.886166,-12.061443,...,5.762778,5.638466,5.615257,5.700178,5.614255,6.078587,5.028918,6.030809,5.190070,1
2,-29.227987,-17.512156,-12.889992,-10.444262,-6.002658,-3.758135,-7.804768,-13.077175,-13.553844,-8.826625,...,6.766705,5.085991,5.169832,5.009525,6.149023,6.494514,6.007160,5.493922,6.129125,1
3,-29.815071,-19.303326,-15.205947,-15.926250,-11.458158,-6.659806,-8.331448,-16.424688,-17.918730,-14.318838,...,6.620479,5.937672,5.978293,5.610490,4.562005,5.695933,5.273780,6.203687,5.690260,1
4,-30.802912,-18.895006,-14.374621,-15.734864,-12.035415,-3.611909,-0.110394,-8.628514,-13.900740,-13.272698,...,6.247078,5.390952,5.733102,5.819227,6.232714,6.432842,7.239798,6.393493,5.631838,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-12.401146,-0.863785,3.488235,5.066339,8.006362,6.627263,5.438753,6.943281,5.270898,5.867504,...,5.088542,5.474295,7.001912,5.205812,5.387005,5.467983,5.312145,5.898367,6.157901,1
1996,-27.323248,-15.428140,-12.453194,-8.608556,-2.639188,-4.457172,-8.821723,-9.822476,-12.290321,-9.429913,...,6.019198,5.395853,5.611296,4.527742,6.294105,6.059382,5.401274,5.964453,6.650590,1
1997,-13.743665,-1.876593,3.717991,5.539287,7.642332,9.268235,8.393232,6.106929,6.694256,6.832345,...,6.154174,6.392594,6.198972,6.813655,7.277905,6.183849,6.519565,5.920331,6.294313,1
1998,-27.758062,-18.928467,-14.702780,-11.927574,-4.733092,-3.876189,-10.560834,-12.152486,-13.107001,-9.204597,...,5.927179,5.520580,4.686990,7.021891,6.517790,5.693713,6.179589,5.579060,5.301518,1


## Not Queen Test

In [12]:
folder_path = 'D:/Hoc Tap/Python/KY4/AIL303m/data/20k_audio_splitted_dataset-20240201T051135Z-001/20k_audio_splitted_dataset/test/NonQueen/'

wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]

df_merge_test_not_queen = pd.DataFrame()
df_merge_test_not_queen_mean= pd.DataFrame()
df_merge_test_not_queen_std= pd.DataFrame()

for wav_file in wav_files:
    file_path = os.path.join(folder_path, wav_file)
    
    y, sr = librosa.load(file_path)
    stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    magnitude = np.abs(stft_result)
    magnitude_db = librosa.amplitude_to_db(magnitude)

    df = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df2 = pd.DataFrame(magnitude_db[Begin_bin:frequency_bins, :]) 
    df_trans = df.std(axis=1).to_frame().T
    df_trans2 = df2.mean(axis=1).to_frame().T
    
    df_merge_test_not_queen_mean = pd.concat([df_merge_test_not_queen_mean,df_trans2], axis=0, ignore_index=True)
    df_merge_test_not_queen_std = pd.concat([df_merge_test_not_queen_std,df_trans], axis=0, ignore_index=True)
    df_merge_test_not_queen = pd.concat([df_merge_test_not_queen_mean, df_merge_test_not_queen_std], axis=1, ignore_index=True)

df_merge_test_not_queen['Label'] = 0
df_merge_test_not_queen


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-14.303803,-3.203200,2.690853,4.447613,3.565811,6.533846,7.883011,4.297709,3.794354,5.225731,...,5.072681,5.128530,6.080802,5.109303,4.311131,5.485855,5.748446,5.507449,5.905296,0
1,-9.373874,0.136103,4.313168,6.741927,6.688936,7.294606,6.579124,3.853055,5.325548,8.108232,...,5.893367,5.253557,6.338323,4.921704,5.668043,5.785707,5.537985,5.907280,5.879009,0
2,-29.275913,-19.976349,-15.294355,-10.506942,-6.951826,-8.924492,-12.384725,-13.875593,-16.160372,-15.848764,...,5.747758,6.407538,5.260863,4.355821,5.806941,6.247874,6.196814,5.012671,4.675068,0
3,-30.046743,-22.447519,-17.609814,-13.359888,-9.026911,-7.305490,-13.875810,-16.287037,-16.946159,-13.561595,...,5.908942,5.409426,5.767831,5.702452,6.002881,5.294115,6.324986,5.548191,4.674161,0
4,-27.193817,-16.962814,-14.548822,-11.893677,-11.900272,-14.232656,-9.377313,-12.141542,-17.741560,-14.992158,...,5.622072,5.614020,5.065368,5.955492,6.294818,6.324538,5.865729,5.701810,5.793783,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-30.204502,-21.420630,-15.907965,-10.847192,-5.540770,-5.376230,-8.865355,-10.644891,-10.147081,-8.459474,...,5.617641,5.518352,5.436143,6.610126,5.798985,4.641056,5.499515,4.662755,5.257440,0
1996,-30.181795,-20.415623,-14.869097,-9.778924,-2.269079,-3.610411,-10.345087,-12.243808,-10.501235,-8.942733,...,5.688550,6.031071,5.379608,6.091762,5.445347,5.152103,5.775897,5.465519,5.647381,0
1997,-28.775091,-19.560608,-15.048741,-10.619410,-7.233252,-7.028648,-7.715782,-10.892303,-12.823707,-9.606281,...,5.561561,6.231637,4.852855,5.094002,5.685498,6.785153,5.303559,5.123080,5.341612,0
1998,-28.311047,-18.685659,-15.497118,-8.910604,-2.277527,-4.192913,-6.853292,-7.389444,-8.578888,-6.382261,...,6.630333,6.800620,6.668313,7.137140,6.539038,7.543288,5.765469,6.601450,7.650997,0


## Final Dataset

In [13]:
df_training = pd.concat([df_merge_train_queen,df_merge_train_not_queen,df_merge_val_not_queen,df_merge_val_queen],axis=0, ignore_index=True)
train_set = df_training.sample(frac=1, random_state=42).reset_index(drop=True)
train_set

,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-30.396360,-21.852654,-18.098604,-16.045698,-15.361418,-13.537639,-8.221273,-9.820609,-16.732294,-14.737292,...,5.502463,5.028520,6.453796,5.170168,7.261175,5.037125,5.409114,5.633767,5.332738,0
1,-25.300264,-19.281168,-17.019773,-18.552090,-13.021339,-11.038358,-16.643486,-18.765831,-22.854675,-18.046814,...,6.410394,6.084327,6.186674,6.508203,6.136508,6.050554,5.782176,5.754456,4.955972,1
2,-11.807796,-0.346278,4.467739,6.242795,10.071224,8.755786,8.048417,7.970694,4.861415,6.647508,...,5.971067,5.747733,6.458572,5.440312,5.038877,6.533874,5.201336,5.146182,6.870896,1
3,-27.251751,-17.711775,-9.271448,-8.506715,-4.951308,0.467168,-5.093977,-9.580098,-8.791265,-5.583716,...,5.116714,4.679895,5.682612,4.802626,5.265350,6.153585,6.507027,5.940607,5.598474,1
4,-25.771212,-16.285900,-11.726841,-10.152008,-7.109600,-8.543331,-6.276131,-0.236455,-3.994293,0.826839,...,6.440270,6.496203,7.365578,6.536973,8.413149,7.010584,6.625439,6.042202,7.062164,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,-10.555495,-2.429872,2.179510,3.745017,4.117899,4.872036,5.756128,6.821715,4.007398,5.939843,...,6.117005,7.106951,5.331597,6.875189,4.627927,4.927350,5.301602,4.250177,4.496144,0
15996,-27.581928,-15.454248,-10.447031,-8.530722,-1.964501,-4.415997,-15.440039,-16.306482,-15.120822,-12.984470,...,6.057472,6.498744,6.775319,6.226325,7.114686,6.863910,5.968043,5.853517,5.112450,1
15997,-25.036068,-13.833079,-11.086799,-6.366263,2.198174,1.271387,-6.235985,-9.231989,-6.539805,-6.808390,...,10.321971,10.164030,9.619773,10.574327,11.323029,10.190562,10.308031,10.616125,9.858575,1
15998,-14.377069,-2.129603,2.631946,4.904552,8.536654,9.819491,7.048007,6.358101,7.320966,10.695164,...,7.406477,6.185484,5.889271,4.687744,5.452734,6.119667,5.846716,5.551431,5.854378,1


In [14]:
df_testing = pd.concat([df_merge_test_queen,df_merge_test_not_queen],axis=0, ignore_index=True)
test_set= df_testing.sample(frac=1, random_state=42).reset_index(drop=True)
test_set


,0,1,2,3,4,5,6,7,8,9,...,215,216,217,218,219,220,221,222,223,Label
0,-27.915800,-18.727110,-14.745368,-11.537800,-8.318852,-8.147633,-12.713858,-17.642008,-18.131641,-17.319168,...,5.528016,5.125944,5.328723,5.353093,4.826367,5.404289,5.506860,6.754473,6.200127,1
1,-31.878803,-24.390692,-21.274332,-20.356167,-17.027058,-12.331870,-14.729094,-21.044836,-15.368110,-9.718509,...,6.290085,5.339860,6.188074,4.512528,4.832645,6.086148,6.759972,5.223155,6.078615,0
2,-28.039040,-18.854761,-13.949584,-8.107493,-4.931336,2.730219,-0.845369,-11.499155,-13.773221,-10.799647,...,6.810291,6.337001,5.870383,5.939550,4.880335,5.750951,6.277734,5.173925,4.419739,1
3,-29.406088,-17.719580,-13.888568,-8.920301,-5.847449,-6.981443,-10.460831,-13.133369,-12.656227,-10.720214,...,5.779501,5.540891,5.853489,5.757949,5.954951,5.585614,5.431163,6.293993,6.608383,0
4,-24.428644,-14.749826,-9.177162,-6.180633,-4.933303,-10.022043,-11.578983,-8.171264,-9.234610,-16.161594,...,7.095370,7.006038,6.554459,5.332487,5.060679,6.231162,5.465745,5.813043,5.436113,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-29.329191,-20.439762,-16.109333,-12.137665,-7.353946,-7.341989,-9.533638,-12.535625,-10.341348,-10.039239,...,5.893899,6.074894,5.554789,5.771777,6.472948,5.621775,5.488806,5.689650,5.535917,1
3996,-27.202047,-18.730545,-14.894341,-11.481274,-7.315171,-6.421489,-10.040202,-10.505864,-10.418355,-6.141046,...,6.404579,5.728745,5.053908,4.937921,6.091786,5.188753,4.172692,5.923851,5.649449,1
3997,-28.123451,-20.497892,-17.701632,-16.397404,-13.316484,-12.585991,-11.690138,-14.645185,-16.922779,-14.796307,...,6.207689,5.380175,5.941535,6.447760,5.814198,5.893470,4.656613,4.881429,5.425532,1
3998,-3.714732,5.062189,11.164470,13.305595,13.207368,12.989984,11.734979,10.804133,9.281901,8.735717,...,5.742276,4.791704,5.284997,5.541826,4.366748,5.913747,6.880213,3.965556,6.223191,0


# Normalization

In [15]:
y_train = train_set['Label']
X_train = train_set.drop(['Label'], axis=1)
y_test = test_set['Label']
X_test = test_set.drop(['Label'], axis=1)

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_test = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test)

# RF

In [17]:
start_time = time.time()

RF_model = RandomForestClassifier()

RF_model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Time Train: {train_time}s")

Time Train: 65.64636397361755s


In [18]:
start_time = time.time()

print('ACC: ',RF_model.score(X_test, y_test))

test_time = time.time() - start_time
print(f"Time Test: {test_time}s")

ACC:  0.9425
Time Test: 0.08921384811401367s


#   ET

In [19]:
start_time = time.time()

ET_model = ExtraTreesClassifier()
ET_model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Time Train: {train_time}s")

Time Train: 6.298863410949707s


In [20]:
start_time = time.time()

print ('ACC: ',ET_model.score(X_test, y_test))

test_time = time.time() - start_time
print(f"Time Test: {test_time}s")

ACC:  0.9415
Time Test: 0.12503933906555176s


#   KNN

In [21]:
start_time = time.time()

KNN_model = KNeighborsClassifier()
KNN_model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Time Train: {train_time}s")

Time Train: 0.014078855514526367s


In [22]:
start_time = time.time()

print ('ACC: ',KNN_model.score(X_test, y_test))

test_time = time.time() - start_time
print(f"Time Test: {test_time}s")

ACC:  0.83625
Time Test: 1.2146775722503662s


#   SVM

In [23]:
start_time = time.time()

SVM_model = SVC()
SVM_model.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Time Train: {train_time}s")

Time Train: 26.575608491897583s


In [24]:
start_time = time.time()

print ('ACC: ',SVM_model.score(X_test, y_test))

test_time = time.time() - start_time
print(f"Time Test: {test_time}s")

ACC:  0.90225
Time Test: 7.1653971672058105s


#   Logistic Regression

In [25]:
start_time = time.time()

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

train_time = time.time() - start_time
print(f"Time Train: {train_time}s")

Time Train: 0.8386743068695068s


c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
start_time = time.time()

print ('ACC: ',log_reg.score(X_test, y_test))

test_time = time.time() - start_time
print(f"Time Test: {test_time}s")

ACC:  0.8465
Time Test: 0.006968259811401367s
